# Advanced settings for TRACE:  Speedups for large-N systems

Similar to WHFAST (see [AdvWHFast.ipynb](../AdvWHFast)), TRACE has a "safe mode" setting that is on by default, but can be turned off for a speed boost. The specific speedups detailed here are **only** useful for systems with large N -- around $N \gtrsim 100$.

**TL;DR**

As long as you don't add, remove or otherwise modify particles between timesteps, you can set `sim.ri_trace.safe_mode = 0` for up to a $\%$ performance boost. If you want to modify particles, or if the code breaks with these advanced settings, read below for details.

**The TRACE algorithm**

The speedup from turning off safe mode come from reducing the computational overhead associated with the various checks used in the TRACE algorithm. Here we briefly walk through the algorithm -- see Lu, Hernandez & Rein (2024) for more details. In one TRACE step:

1. Every pair of particles is checked for close encounters, a mutual separation criterion informed by each particle's Hill radius.

2. The integration itself happens, with either Wisdom-Holman or Bulirsch-Stoer depending on close encounters

3. Every pair of particles is checked again for close encounters. If any pair of particles initially not in a close encounter is in one after the timestep, the step is rejected. We then re-run the step, demanding that the new particle pair is integrated with the close encounter prescription.

All three steps scale as $O(N^2)$ where $N$ is particle number, and it turns out that in the limit of large $N$ the overhead from checks (Steps 1 & 3) actually dominates over the actual integration time (Steps 2). 

For a pure $N$-body code there is no getting around this $O(N^2)$ bottleneck, but one thing we can do is recycle the post-timestep close encounter check. In the event that after Step 3:

* The step is not rejected
* No particles are changed between timesteps

Then the simulation state at the end of Step 3 is exactly the same as the state at the beginning of Step 1 of the next timestep. Step 1 is thus a redundant calculation, and we can safely skip it provided the two conditions above are met.

**Overriding the defaults**

Let's begin by importing rebound, and defining a simple function to reset rebound and initialize a new simulation with a large $N$. Specifically, we use a protoplanetary disk inspired by Chambers (2013).

In [3]:
import rebound
import numpy as np
import time

# 14 embryos, 140 planetesimals
embryo_smas = [3.75e-01, 4.83e-01, 7.00e-01, 7.55e-01, 8.01e-01, 8.71e-01, 9.37e-01, 1.01e+00, 1.02e+00, 1.06e+00,
               1.12e+00, 1.13e+00, 1.33e+00, 1.56e+00]

planetesimal_smas = [0.306,0.307,0.312,0.338,0.344,0.349,0.366,0.375,0.378,0.392,0.396,0.404,0.416,0.435,0.438,
                     0.442,0.448,0.494,0.505,0.512,0.512,0.548,0.549,0.605,0.701,0.713,0.718,0.727,0.730,0.737,
                     0.741,0.744,0.745,0.747,0.748,0.748,0.761,0.761,0.762,0.767,0.771,0.772,0.773,0.778,0.779,
                     0.780,0.799,0.799,0.817,0.819,0.822,0.828,0.837,0.846,0.854,0.873,0.875,0.875,0.882,0.890,
                     0.892,0.892,0.900,0.905,0.908,0.911,0.919,0.933,0.946,0.949,0.955,0.977,0.979,0.980,0.981,
                     0.983,0.984,0.991,1.014,1.019,1.022,1.049,1.053,1.069,1.070,1.074,1.089,1.099,1.133,1.162,
                     1.188,1.195,1.254,1.258,1.265,1.278,1.280,1.288,1.298,1.311,1.325,1.325,1.338,1.339,1.353,
                     1.356,1.358,1.363,1.370,1.373,1.420,1.425,1.453,1.468,1.488,1.498,1.502,1.504,1.528,1.553,
                     1.571,1.598,1.617,1.641,1.687,1.689,1.717,1.730,1.731,1.753,1.811,1.831,1.837,1.861,1.882,
                     1.917,1.927,1.934,1.959,1.968]

def test_case():
    sim = rebound.Simulation()
    sim.integrator = "trace"
    sim.add(m=1) # Star
    
    # Add embryos based on semimajor axis, randomizing other orbital elements
    for i, em_sma in enumerate(embryo_smas):
        sim.add(m=2.8e-7,a=em_sma,e=np.random.rand()*0.1,inc=np.random.rand()*0.0175,
               omega=np.random.rand()*2*np.pi,Omega=np.random.rand()*2*np.pi,f=np.random.rand()*2*np.pi)
    
    # And planetesimals
    for i, pl_sma in enumerate(planetesimal_smas):
        sim.add(m=2.8e-8,a=pl_sma,e=np.random.rand()*0.1,inc=np.random.rand()*0.0175,
               omega=np.random.rand()*2*np.pi,Omega=np.random.rand()*2*np.pi,f=np.random.rand()*2*np.pi)
        
    # Finally, Jupiter and Saturn
    sim.add(m=9.543e-4, a=5.2)
    sim.add(m=2.857e-4, a=9.5)
    sim.move_to_com()
    sim.dt = 6./365.*2*np.pi
    return sim

In [4]:
sim = test_case()
start_time = time.time()
sim.integrate(1e4*2*np.pi)
print("Safe integration took {0} seconds".format(time.time() - start_time))

sim = test_case()
sim.ri_trace.safe_mode = 0
start_time = time.time()
sim.integrate(1e4*2*np.pi)
print("Manual integration took {0} seconds".format(time.time() - start_time))

Safe integration took 171.52959489822388 seconds
Manual integration took 116.15049505233765 seconds
